In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import pickle
from tqdm import tqdm

sns.set_theme("notebook")

In [ ]:
RESULTS_DIR = Path("/data5/anasynth_nonbp/andre/reverse-dj-mix/results")
UNMIXDB_PATH = Path("/data2/anasynth_nonbp/schwarz/abc-dj/data/unmixdb-zenodo")

In [ ]:
results = []
for i in tqdm(RESULTS_DIR.glob("*/*/results.pickle")):
    with open(i, "rb") as f:
        data = pickle.load(f)
        data["run"] = i.parent.parent.name
        results.append(data)

In [ ]:
data = (
    pd.json_normalize(results)
)
cols = [
    "fadeout_start.err",
    "fadeout_stop.err",
    "fadein_start.err",
    "fadein_stop.err",
    "track_start.err",
    "speed.err",
]

for col in cols:
    data[col] = data[col].apply(lambda x: np.mean(x))

data = data.apply(pd.to_numeric, errors="ignore")
data[["stretch", "fx"]] = data["mix_name"].str.extract(r"set\d+mix\d+-(\w+)-(\w+)-\d+")
data["stretch"] = data["stretch"].str.slice(0, 5)
data["fx"] = data["fx"].str.slice(0, 5)
data["transformation"] = data["stretch"] + " " + data["fx"]
data

In [ ]:
sns.boxplot(data, x="transformation", y="gain.err", hue="run", log_scale=True)
plt.xticks(rotation=90)
plt.show()

In [ ]:
sns.boxplot(data, x="transformation", y="warp.err", hue="run", log_scale=True)
plt.xticks(rotation=90)
plt.show()

In [ ]:
for i in cols:
    sns.boxplot(data, x="transformation", y=i, hue="run", log_scale=True)
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
data["fadeout_start.err"]